In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/cisc867/project/

/content/drive/MyDrive/cisc867/project


In [3]:
!git init

Initialized empty Git repository in /content/drive/MyDrive/cisc867/project/.git/


In [6]:
!git remote add origin https://github.com/emedema/id_humpback_whales

In [9]:
!git pull origin main

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 5 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/emedema/id_humpback_whales
 * branch            main       -> FETCH_HEAD
 * [new branch]      main       -> origin/main


In [11]:
!git config --global user.email "medema.emily@gmail.com"
!git config --global user.name "emedema"

In [13]:
!git add .

In [14]:
!git commit -m "notebook and git setup"

[master f4fda41] notebook and git setup
 1 file changed, 1 insertion(+)
 create mode 100644 humpback_whale_preprocess.ipynb


In [25]:
!git push --set-upstream origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [26]:
!git remote -v

origin	https://github.com/emedema/id_humpback_whales (fetch)
origin	https://github.com/emedema/id_humpback_whales (push)
